In [2]:
import pandas as pd
from sqlalchemy import create_engine

# Set up the database connection
server_name = '----'
database_name = '-----'
username = ''
password = ''

connection_string = f'mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)

# Define your SQL query
sql_query = """
    SELECT
        order_id,
        category_name,
        brand_name,
        product_name,
        quantity,
        i.list_price
    FROM
        sales.order_items i
        JOIN production.products p ON p.product_id = i.product_id
        JOIN production.categories c ON c.category_id = p.category_id
        JOIN production.brands b ON b.brand_id = p.brand_id
"""

# Fetch data from the database and load it into a DataFrame
df = pd.read_sql_query(sql_query, engine)

# Print the first few rows of the DataFrame
df


,order_id,category_name,brand_name,product_name,quantity,list_price
0,1,Cruisers Bicycles,Electra,Electra Townie Original 7D EQ - Women's - 2016,1,599.99
1,1,Mountain Bikes,Trek,Trek Remedy 29 Carbon Frameset - 2016,2,1799.99
2,1,Cyclocross Bicycles,Surly,Surly Straggler - 2016,2,1549.00
3,1,Cruisers Bicycles,Electra,Electra Townie Original 7D EQ - 2016,2,599.99
4,1,Mountain Bikes,Trek,Trek Fuel EX 8 29 - 2016,1,2899.99
...,...,...,...,...,...,...
4640,1614,Road Bikes,Trek,Trek Emonda ALR 6 - 2018,2,2299.99
4641,1614,Cruisers Bicycles,Electra,Electra Cruiser 1 - 2016/2017/2018,2,269.99
4642,1615,Electric Bikes,Trek,Trek Verve+ Lowstep - 2018,2,2299.99
4643,1615,Cruisers Bicycles,Electra,Electra Tiger Shark 3i - 2018,1,899.99


In [14]:
import os
import pandas as pd
from sqlalchemy import create_engine

# Set up the database connection
server_name = 'DESKTOP-DRB7DGT\DANYA'
database_name = 'bikestores'
username = ''
password = ''

connection_string = f'mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)

# Define your SQL query
sql_query = """
    SELECT
        order_id,
        category_name,
        brand_name,
        product_name,
        quantity,
        i.list_price
    FROM
        sales.order_items i
        JOIN production.products p ON p.product_id = i.product_id
        JOIN production.categories c ON c.category_id = p.category_id
        JOIN production.brands b ON b.brand_id = p.brand_id
"""

# Fetch data from the database and load it into a DataFrame
df = pd.read_sql_query(sql_query, engine)

# Specify the folder to save Excel files
output_folder = r'C:\Users\danis\Desktop\Excel Files'

# Create Excel files with multiple sheets
category_groups = df.groupby('category_name')

for category_name, group_df in category_groups:
    excel_file = os.path.join(output_folder, f'{category_name}_data.xlsx')
    writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
    
    brand_groups = group_df.groupby('brand_name')
    for brand_name, brand_group_df in brand_groups:
        brand_group_df.to_excel(writer, sheet_name=brand_name, index=False)
    
    writer.save()


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\danis\\Desktop\\Excel Files\\Mountain Bikes_data.xlsx'

In [13]:
import os
import pandas as pd
from sqlalchemy import create_engine

# Set up the database connection
server_name = 'DESKTOP-DRB7DGT\DANYA'
database_name = 'bikestores'
username = ''
password = ''

connection_string = f'mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)

# Define your SQL query
sql_query = """
    SELECT
        order_id,
        category_name,
        brand_name,
        product_name,
        quantity,
        i.list_price
    FROM
        sales.order_items i
        JOIN production.products p ON p.product_id = i.product_id
        JOIN production.categories c ON c.category_id = p.category_id
        JOIN production.brands b ON b.brand_id = p.brand_id
"""

# Fetch data from the database and load it into a DataFrame
df = pd.read_sql_query(sql_query, engine)

# Specify the folder to save Excel files
output_folder = r'C:\Users\danis\Desktop\Excel Files'

# Create an audit DataFrame to track the imports
audit_data = []

# Loop through the Excel files and append new data if not already present
category_groups = df.groupby('category_name')

for category_name, group_df in category_groups:
    excel_file = os.path.join(output_folder, f'{category_name}_data.xlsx')
    existing_df = {}
    
    if os.path.exists(excel_file):
        existing_df = pd.read_excel(excel_file, sheet_name=None, engine='openpyxl')
    
    brand_groups = group_df.groupby('brand_name')
    for brand_name, brand_group_df in brand_groups:
        sheet_name = brand_name
        if category_name in existing_df and sheet_name in existing_df[category_name]:
            existing_rows = existing_df[category_name][sheet_name].set_index(['order_id', 'product_name']).index
            new_rows = brand_group_df[
                ~brand_group_df.set_index(['order_id', 'product_name']).index.isin(existing_rows)
            ]
            
            if not new_rows.empty:
                existing_df[category_name][sheet_name] = pd.concat([existing_df[category_name][sheet_name], new_rows])
        else:
            if category_name not in existing_df:
                existing_df[category_name] = {}
            existing_df[category_name][sheet_name] = brand_group_df.copy()

    # Write all the sheets to the Excel file
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        for category, sheet_data in existing_df.items():
            for sheet_name, data in sheet_data.items():
                data.to_excel(writer, sheet_name=sheet_name, index=False)
    
    # Append audit data
    audit_data.extend([
        {
            'ID': i,
            'file_name': excel_file,
            'sheet_name': sheet_name,
            'num_rows_imported': len(data)
        }
        for i, (sheet_name, data) in enumerate(existing_df.get(category_name, {}).items(), start=1)
    ])

# Create the audit DataFrame
audit_df = pd.DataFrame(audit_data)

# Specify the path for the audit Excel file
audit_excel_file = os.path.join(output_folder, 'audit.xlsx')

# Write the audit DataFrame to an Excel file
audit_df.to_excel(audit_excel_file, index=False)

print("Audit data saved to Excel file:", audit_excel_file)


Audit data saved to Excel file: C:\Users\danis\Desktop\Excel Files\audit.xlsx
